# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket 
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

Solve the problem by hand, using tree search. What solutions have you found?

### Buildind a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [51]:
class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2
    
    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.b1, self.b2)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

Now we define each of the operators on states:

In [52]:
# emptying the first bucket
def empty1(state):
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# emptying the second bucket
def empty2(state):
    if state.b2 > 0:
        return BucketState(state.b1,0)
    return None

def fill1(state):
    if state.b1 < BucketState.c1:
        return BucketState(BucketState.c1,state.b2)
    return None

def fill2(state):
    if state.b2 < BucketState.c2:
        return BucketState(state.b1,BucketState.c2)
    return None

def pour12_fill2(state):
    if state.b1 > 0 and state.b2 < BucketState.c2 and state.b1  >= BucketState.c2 - state.b2:
        return BucketState(state.b1 - (BucketState.c2 - state.b2),BucketState.c2)
    return None

def pour12_empty1(state):
    if state.b1 > 0 and state.b2 < BucketState.c2 and state.b1 < BucketState.c2 - state.b2:
        return BucketState(0,state.b2 + state.b1)
    return None

def pour21_fill1(state):
    if state.b2 > 0 and state.b1 < BucketState.c1 and state.b2 >= BucketState.c1 - state.b1:
        return BucketState(BucketState.c1,state.b2 - (BucketState.c1 - state.b1))
    return None

def pour21_empty2(state):
    if state.b2 > 0 and state.b1 < BucketState.c1 and state.b2 < BucketState.c1 - state.b1:
        return BucketState(state.b2 + state.b1,0)
    return None
# your code here


The following function will aggregate all states that can be generated from a given one:

In [53]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append(empty1(state))
    if(empty2(state)):
        new_states.append(empty2(state))
    if(fill1(state)):
        new_states.append(fill1(state))
    if(fill2(state)):
        new_states.append(fill2(state))
    if(pour12_fill2(state)):
        new_states.append(pour12_fill2(state))
    if(pour12_empty1(state)):
        new_states.append(pour12_empty1(state))
    if(pour21_fill1(state)):
        new_states.append(pour21_fill1(state))
    if(pour21_empty2(state)):
        new_states.append(pour21_empty2(state))
    return new_states

Play around with the state transition operators and check if they are working properly:

In [54]:
s = BucketState(0, 0)
s = fill1(s)


states = child_bucket_states(BucketState(4, 0))
for i in states:
    print(i)

# your code here


(0, 0)
(4, 3)
(1, 3)


Finally, we need to define the goal condition:

In [55]:
def goal_bucket_state(state):
    if(state.b1 == 2):
        return True
    return False

Test your goal condition:

In [56]:
print(goal_bucket_state(BucketState(0,0)))
print(goal_bucket_state(BucketState(2,0)))
print(goal_bucket_state(BucketState(4,0)))


False
True
False


#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [57]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [58]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])
    visited = set([initial_state])   # initialize the queue to store the nodes
    
    while queue:
        node = queue.popleft()  # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node
        
        for state in operators_func(node.state):
            if state not in visited:   # go through next states
                child = TreeNode(state=state, parent=node)
                node.add_child(child)
                queue.append(child)
                visited.add(state)

    return None

We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [59]:
goal = breadth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)

(2, 3)


In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [60]:
def print_solution(node):
    lista = [node.state]
    while(node.parent):
        lista.append(node.parent.state)
        node = node.parent
    for i in reversed(lista):
        print(i)
    return

Now we can print the solution:

In [61]:
print_solution(goal)

(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [62]:
from collections import deque
def depth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])
    visited = set([initial_state])   # initialize the queue to store the nodes
    
    while queue:
        node = queue.pop()  # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node

        for state in operators_func(node.state):
            if state not in visited: # go through next states
                child = TreeNode(state=state, parent=node)
                node.add_child(child)
                queue.append(child)
                visited.add(state)
                
    return None 

Test it on the two buckets problem.

In [63]:
goal = depth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print_solution(goal)

(0, 0)
(0, 3)
(3, 0)
(3, 3)
(4, 2)
(0, 2)
(2, 0)


If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [64]:
#this can also be done by adding an attribute depth to the class, and if parent != None then depth = parent.depth + 1

def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([(root,0)])
    visited = set([initial_state])   # initialize the queue to store the nodes
    
    while queue:
        node, depth = queue.pop()
        if goal_state_func(node.state):   # check goal state
            return node
        for state in operators_func(node.state):
            if state not in visited and (depth + 1 <= depth_limit): # go through next states
                child = TreeNode(state=state, parent=node)
                node.add_child(child)
                queue.append((child,depth+1))
                visited.add(state)
                
    return None   # your code here
    

Test it on the two buckets problem.

In [65]:
goal = depth_limited_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states,6)

if goal == None:
    print("No solution found within depth limit")
else:
    print_solution(goal)


(0, 0)
(0, 3)
(3, 0)
(3, 3)
(4, 2)
(0, 2)
(2, 0)


##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [66]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func, depth_limit):
    for i in range(depth_limit):
        goal = depth_limited_search(BucketState(0,0), goal_bucket_state, child_bucket_states,i)
        if(goal != None):
            print(f'Solution in {i} steps')
            return goal
    return None

Again, test it on the two buckets problem.

In [67]:
# your code here
goal = iterative_deepening_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states,8)

if goal == None:
    print("No solution found within depth limit")
else:
    print_solution(goal)

Solution in 6 steps
(0, 0)
(0, 3)
(3, 0)
(3, 3)
(4, 2)
(0, 2)
(2, 0)


## The Missionaries and Cannibals Problem 

<img src="https://allfish24.files.wordpress.com/2016/09/gambar-1.jpg">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

~ Being N the initial number of Missionaires / Cannibals

State:
[M / C  / B] M: 0..N & C: 0..N & B: 0..1 => right side of river 
[N - M / N - C / abs(B - 1)] left side of the river

Initial State:
[3/3/1] on first bank

Objective State:
[0/0/_]

Operators:                              Preconditions:                                                                    Effects:
Move 1 Missionaire                      M - 1 >= C or M - 1 == 0, M > 0,(N - M + 1) >= (N - C)                            M = M - 1
Move 1 Cannibal                         M >= C - 1, C > 0, (N - M) >= (N - C + 1) or N - M == 0                           C - 1
Move 2 Missionaires                     M - 2 >= C, M > 1,(N - M + 2) >= (N - C)                                          M = M- 2
Move 2 Cannibals                        M >= C - 2, C > 1,(N - M ) >= (N - C + 2)                                         C = C - 2
Move 1 Missionaire & 1 Cannibal         M - 1 >= C -1, M > 0, C > 0, (N - M + 1) >= (N - C + 1)                           M = M - 1 & C = C - 1                                           

In [68]:
class RiverState:
    num = 3  # initial nr of missionaires/cannibals
    
    def __init__(self, m1, c1,b):
        self.m1 = m1
        self.c1 = c1
        self.b = b

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.m1, self.c1,self.b)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.m1) + ", " + str(self.c1) + ", " + str(self.b) + ")"

def move1Miss(state):
    m2 = state.num - state.m1
    c2 = state.num - state.c1
    if (state.b == 1) and (state.m1 > 0) and (state.m1 - 1 >= state.c1) and (m2 + 1 >= c2):
        return RiverState(state.m1 - 1,state.c1,0)
    if (state.b == 0) and (m2 > 0) and (m2 - 1 >= c2) and (state.m1 + 1 >= state.c1):
        return RiverState(state.m1 + 1,state.c1,1)
    return None

def move1Cannibal(state):
    m2 = state.num - state.m1
    c2 = state.num - state.c1
    if(state.b == 1) and (state.c1 > 0) and (state.m1 >= state.c1 - 1 or state.m1 == 0) and (m2 >= c2 + 1 or m2 == 0):
         return RiverState(state.m1,state.c1-1,0)
    if(state.b == 0) and (c2 > 0) and (m2 >= c2 - 1 or m2 == 0) and (state.m1 >= state.c1 + 1 or state.m1 == 0):
         return RiverState(state.m1,state.c1+1,1)
    return None

def move2Cannibal(state):
    m2 = state.num - state.m1
    c2 = state.num - state.c1
    if(state.b == 1) and (state.c1 > 1) and (state.m1 >= state.c1 - 2 or state.m1 == 0) and (m2 >= c2 + 2 or m2 == 0):
         return RiverState(state.m1,state.c1-2,0)
    if(state.b == 0) and (c2 > 1) and (m2 >= c2 - 2 or m2 == 0) and (state.m1 >= state.c1 + 2 or state.m1 == 0):
         return RiverState(state.m1,state.c1+2,1)
    return None

def move1Missionaire(state):
    m2 = state.num - state.m1
    c2 = state.num - state.c1
    if(state.b == 1) and state.m1 > 0 and (state.m1 - 1 >= state.c1 or state.m1 - 1 == 0) and (m2 + 1 >= c2):
        return RiverState(state.m1 - 1,state.c1,0)
    if(state.b == 0) and m2 > 0 and (m2 - 1 >= c2 or m2 - 1 == 0) and (state.m1 + 1 >= state.c1):
        return RiverState(state.m1 + 1,state.c1,0)

def move2Missionaire(state):
    m2 = state.num - state.m1
    c2 = state.num - state.c1
    if(state.b == 1) and state.m1 > 1 and (state.m1 - 2 >= state.c1 or state.m1 - 2 == 0) and (m2 + 2 >= c2):
        return RiverState(state.m1 - 2,state.c1,0)
    if(state.b == 0) and m2 > 1 and (m2 - 2 >= c2 or m2 - 2 == 0) and (state.m1 + 2 >= state.c1):
        return RiverState(state.m1 + 2,state.c1,0)

def moveBoth(state):
    m2 = state.num - state.m1
    c2 = state.num - state.c1
    if(state.b == 1) and state.m1 > 0 and state.c1 > 0 and (state.m1 >= state.c1 or state.m1 - 1 == 0) and (m2 + 1 >= c2 + 1): 
        return RiverState(state.m1 - 1,state.c1 - 1,0)
    if(state.b == 0) and m2 > 1 and c2 > 1 and (m2 >= c2 or m2 - 1 == 0) and (state.m1 + 1 >= state.c1 + 1):
         return RiverState(state.m1 + 1,state.c1 + 1,1)

def child_river_states(state):
    lista = []
    if(move1Cannibal(state)):
        lista.append(move1Cannibal(state))
    if(move2Cannibal(state)):
        lista.append(move2Cannibal(state))
    if(move1Missionaire(state)):
        lista.append(move1Missionaire(state))
    if(move2Missionaire(state)):
        lista.append(move2Missionaire(state))
    if(moveBoth(state)):
        lista.append(moveBoth(state))
    return lista

def goal_river_state(state):
    if(state.m1 == 0 and state.c1 == 0):
        return True
    return False



goal = breadth_first_search(RiverState(3,3,1), 
                            goal_river_state, 
                            child_river_states)
if(goal):
   print_solution(goal)
else:
    print("No solution")


(3, 3, 1)
(3, 1, 0)
(3, 2, 1)
(3, 0, 0)
(3, 1, 1)
(1, 1, 0)
(2, 2, 1)
(0, 2, 0)
(0, 3, 1)
(0, 1, 0)
(0, 2, 1)
(0, 0, 0)


Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [69]:
# your code here
